In [45]:
import psycopg2
import psycopg2.extras
import numpy as np
import pandas as pd

In [46]:
conn = psycopg2.connect(dbname="nyc_taxi_data", user="postgres", password="123456789")

cur = conn.cursor( )

In [47]:
addr_df=pd.read_csv('dataset/yellow_tripdata_2020-01.csv', low_memory=False)
addr_df.reset_index(drop=True,inplace=True)
addr_df.head()
# addr_df_=addr_df.insert(0, 'id', range(1, 1 + len(addr_df)))

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge
0,1.0,2020-01-01 00:28:15,2020-01-01 00:33:03,1.0,1.2,1.0,N,238,239,1.0,6.0,3.0,0.5,1.47,0.0,0.3,11.27,2.5
1,1.0,2020-01-01 00:35:39,2020-01-01 00:43:04,1.0,1.2,1.0,N,239,238,1.0,7.0,3.0,0.5,1.50,0.0,0.3,12.30,2.5
2,1.0,2020-01-01 00:47:41,2020-01-01 00:53:52,1.0,0.6,1.0,N,238,238,1.0,6.0,3.0,0.5,1.00,0.0,0.3,10.80,2.5
3,1.0,2020-01-01 00:55:23,2020-01-01 01:00:14,1.0,0.8,1.0,N,238,151,1.0,5.5,0.5,0.5,1.36,0.0,0.3,8.16,0.0
4,2.0,2020-01-01 00:01:58,2020-01-01 00:04:16,1.0,0.0,1.0,N,193,193,2.0,3.5,0.5,0.5,0.00,0.0,0.3,4.80,0.0


In [48]:
def create_staging_table(cursor):
    cursor.execute("""
        DROP TABLE IF EXISTS nyc_cab_data CASCADE;
        CREATE UNLOGGED TABLE nyc_cab_data (
            vendor_id        INTEGER,
            tpep_pickup_datetime               TIMESTAMP,
            tpep_dropoff_datetime          TIMESTAMP,
            passenger_count           DECIMAL,
            trip_distance       DECIMAL,
            RatecodeID       DECIMAL,
            store_and_fwd_flag       TEXT,
            PULocationID       INTEGER,
            DOLocationID       INTEGER,
            payment_type       DECIMAL,
            fare_amount       DECIMAL,
            extra       DECIMAL,
            mta_tax       DECIMAL,
            tip_amount       DECIMAL,
            tolls_amount       DECIMAL,
            improvement_surcharge       DECIMAL,
            total_amount       DECIMAL,
            congestion_surcharge       DECIMAL
        );""")

In [49]:
with conn.cursor() as cursor:
    create_staging_table(cursor)

In [50]:
def send_csv_to_psql(connection,csv,table_):
    sql = "COPY %s FROM STDIN WITH CSV HEADER DELIMITER AS ','"
    file = open(csv, "r")
    table = table_
    with connection.cursor() as cur:
        cur.execute("truncate " + table + ";")
        cur.copy_expert(sql=sql % table, file=file)
        connection.commit()
    return connection.commit()

send_csv_to_psql(conn,'dataset/yellow_tripdata_2020-01.csv','nyc_cab_data')

In [51]:
def create_id(connection, sql, table_):
    sql = "ALTER TABLE public.nyc_cab_data ADD COLUMN id SERIAL PRIMARY KEY;"
    table = table_
    with connection.cursor() as cur:
        cur.execute(sql)
        connection.commit()
    return connection.commit()
create_id(conn,"ALTER TABLE public.nyc_cab_data ADD COLUMN id SERIAL PRIMARY KEY;",'nyc_cab_data')

In [52]:
sql_="SELECT COUNT(*) FROM nyc_cab_data LIMIT 10;"
cur.execute(sql_)
cur.fetchone()

(6405008,)

In [53]:
cur.execute("SELECT * FROM nyc_cab_data LIMIT 10")
cur.fetchall()

[(1,
  datetime.datetime(2020, 1, 1, 0, 28, 15),
  datetime.datetime(2020, 1, 1, 0, 33, 3),
  Decimal('1'),
  Decimal('1.20'),
  Decimal('1'),
  'N',
  238,
  239,
  Decimal('1'),
  Decimal('6'),
  Decimal('3'),
  Decimal('0.5'),
  Decimal('1.47'),
  Decimal('0'),
  Decimal('0.3'),
  Decimal('11.27'),
  Decimal('2.5'),
  1),
 (1,
  datetime.datetime(2020, 1, 1, 0, 35, 39),
  datetime.datetime(2020, 1, 1, 0, 43, 4),
  Decimal('1'),
  Decimal('1.20'),
  Decimal('1'),
  'N',
  239,
  238,
  Decimal('1'),
  Decimal('7'),
  Decimal('3'),
  Decimal('0.5'),
  Decimal('1.5'),
  Decimal('0'),
  Decimal('0.3'),
  Decimal('12.3'),
  Decimal('2.5'),
  2),
 (1,
  datetime.datetime(2020, 1, 1, 0, 47, 41),
  datetime.datetime(2020, 1, 1, 0, 53, 52),
  Decimal('1'),
  Decimal('0.60'),
  Decimal('1'),
  'N',
  238,
  238,
  Decimal('1'),
  Decimal('6'),
  Decimal('3'),
  Decimal('0.5'),
  Decimal('1'),
  Decimal('0'),
  Decimal('0.3'),
  Decimal('10.8'),
  Decimal('2.5'),
  3),
 (1,
  datetime.datetime